In [2]:
import os
import numpy as np
import pandas as pd
import pysam
import math
import pickle
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# ! ls -l ../../carmelab/backup/genomes/from_Arielle/ust/Ust_Ishim.hg19_1000g.all.bam
# ! pip3 install plotly

## Open, read and parse BAM file and CpG coordinates file:

In [2]:
def load_samfile(sample_name):
    if sample_name.lower() == "altai":
        samfile_path = "../../carmelab/backup/genomes/from_Arielle/altai/altai_bams/Altai_chr21.bam"
    else:
        raise ValueError(f'Unknown sample name: {sample_name}')
    
    samfile = pysam.AlignmentFile(samfile_path, "rb")
    return samfile

In [3]:
with open ("../../carmelab/backup/genomes/cpg_coords.txt") as cpg_coords_file:
    cpg_coords = cpg_coords_file.read().splitlines()
chr21_cpgs = cpg_coords[22].replace("\t", ",").split(",")
chr21_cpgs_dict = {coord: True for coord in chr21_cpgs}

## CpG islands

In [4]:
cpg_df = pd.read_csv("../../carmelab/backup/genomes/CpGs_Islands.csv", index_col=0)
# print(cpg_df.head())
# cpg_df.seqnames.unique()
# print(cpg_df.strand.unique())

In [5]:
def get_distance_from_cpg(chromosome, position):
    distance_from_cpg = math.inf
    cpg_in_chrom = cpg_df.loc[cpg_df['seqnames'] == chromosome]
    #finding nearest CpG island, that starts before position in genome (because after is too late)
    nearest_earlier_island_start_index = np.searchsorted(cpg_in_chrom.start, position , side="right")[0] - 1
    nearest_earlier_island = cpg_in_chrom.iloc[nearest_earlier_island_start_index]
    # if it's end is after the position given, then it's within the island
    if position <= nearest_earlier_island.end:
        distance_from_cpg = 0
    else:
        distance_from_earlier = position - nearest_earlier_island.end
        distance_from_later = math.inf
        if (nearest_earlier_island_start_index + 1) < len(cpg_in_chrom):
            nearest_later_island = cpg_in_chrom.iloc[nearest_earlier_island_start_index + 1]
            distance_from_later = nearest_later_island.start - position
        distance_from_cpg = min(distance_from_earlier, distance_from_later)
                              
    return distance_from_cpg
    
    

# Generate DF of c sites

In [6]:
# ones = 0
# for read in samfile.fetch('21'):
#     if (ones < 1) and (read.is_reverse):
#         if read.mapping_quality < 20 and read.query_length == read.reference_length:
#             read_aligned_pares = read.get_aligned_pairs()
#             read_sequence = read.query_sequence
#             ref_sequence = read.get_reference_sequence()
#             print(read_aligned_pares)
#             for index, base in enumerate(read_sequence):
#                 print(base, ref_sequence[index], index, read.query_length - (read_aligned_pares[index][0] + 1))
#         ones += 1

In [7]:
# count = 0
# nones = 0
# for read in samfile.fetch(chromosome):
#     if read.mapping_quality < 20 and read.query_length == read.reference_length:
#         read_aligned_pares = read.get_aligned_pairs(matches_only=True)
#         read_sequence = read.query_sequence
#         ref_sequence = read.get_reference_sequence()
#         for index, base in enumerate(read_sequence):
#             print(read_aligned_pares[index])
#             if index == read_aligned_pares[index][0] and read.query_alignment_qualities[index] < 53 and read_aligned_pares[index][1] != None and(str(read_aligned_pares[index][1]+1) in chr21_cpgs_dict):
#                 if read_aligned_pares[index][1] is None:
#                     nones+=1
#                 count += 1
                
# print(nones)
# print(count)

In [14]:
df = pd.DataFrame(columns=['read_name', 'position_in_reference', 'position_in_read', 'after', 'distance_from_cpg_island', "letter_before_read", "letter_after_read", "letter_before_ref", "letter_after_ref"])
df_ls = []
samfile = load_samfile("altai")
chromosome = '21'
# samfile.count('21')
reads_processed = 0
for read in samfile.fetch(chromosome, 9998400, 9999500):
    if read.mapping_quality < 20 and read.query_length == read.reference_length:
        read_aligned_pares = read.get_aligned_pairs()
        read_sequence = read.query_sequence
        ref_sequence = read.get_reference_sequence()
        for index, base in enumerate(read_sequence):
            if index == read_aligned_pares[index][0] and read.query_alignment_qualities[index] < 53 and read_aligned_pares[index][1] and (str(read_aligned_pares[index][1]+1) in chr21_cpgs_dict):
                position_in_reference = read_aligned_pares[index][1]
                letter_before_read = read_sequence[index-1] if index > 0 else "!"
                letter_after_read = read_sequence[index+1] if index+1 < len(read_sequence) else "!"
                #TODO - Should capitalize? 
                letter_before_ref = ref_sequence[index-1] if index > 0 else "!"
                letter_after_ref = ref_sequence[index+1] if index+1 < len(read_sequence) else "!"
                new_row = pd.DataFrame([{
                    "read_name":read.query_name,
                    "position_in_reference": position_in_reference,
                    "position_in_read": read_aligned_pares[index][0],
                    "after": base,
                    "distance_from_end": read.query_length - (read_aligned_pares[index][0] + 1),
                    "read_length": read.query_length,
                    "distance_from_cpg_island": get_distance_from_cpg(chromosome, position_in_reference),
                    "letter_before_read": letter_before_read,
                    "letter_after_read": letter_after_read,
                    "letter_before_ref": letter_before_ref,
                    "letter_after_ref": letter_after_ref,
                    "is_cpg_coord": str(position_in_reference+1) in chr21_cpgs_dict,
                    "is_reverse": read.is_reverse
                }])
                df_ls.append(new_row)

    reads_processed += 1
    if reads_processed % 500 == 0:
        print(f"Processed {reads_processed} reads so far")
df = pd.concat(df_ls, axis=0, ignore_index=True)


Processed 500 reads so far
Processed 1000 reads so far
Processed 1500 reads so far
Processed 2000 reads so far
Processed 2500 reads so far


In [15]:
print(df.shape)
df.head()

(309, 13)


,after,distance_from_cpg_island,distance_from_end,is_cpg_coord,is_reverse,letter_after_read,letter_after_ref,letter_before_read,letter_before_ref,position_in_read,position_in_reference,read_length,read_name
0,C,29813,14,True,False,T,g,C,t,148,9998433,163,SN7001204_0131_BC0M3YACXX_PEdi_SS_L9302_L9303_...
1,C,29813,38,True,True,T,g,T,T,147,9998433,186,NIOBE_0139_A_D0B5GACXX:7:1308:16425:75667
2,C,29829,22,True,True,A,g,G,G,163,9998449,186,NIOBE_0139_A_D0B5GACXX:7:1308:16425:75667
3,C,29813,3,True,True,G,G,T,T,140,9998433,144,SN7001204_0131_BC0M3YACXX_PEdi_SS_L9302_L9303_...
4,C,29813,27,True,True,G,G,T,T,120,9998433,148,NIOBE_0139_A_D0B5GACXX:6:2108:9258:134346


In [ ]:
df.to_csv('../../carmelab/backup/genomes/from_Arielle/altai/altai_bams/Altai_chr21.csv')

In [ ]:
df["distance_from_nearest_end"] = df[["position_in_read", "distance_from_end"]].min(axis=1)

In [16]:
df.after.value_counts(normalize=True) * 100

C    92.233010
T     6.472492
G     0.647249
N     0.323625
A     0.323625
Name: after, dtype: float64

In [ ]:
df.letter_after_ref.value_counts()

# Creating Sum Table

In [ ]:
df_ct = df[df["after"].isin(["C", "T"])]
final = df_ct.groupby('after')['distance_from_nearest_end'].value_counts().unstack().fillna(0)
degrad_rate = final.iloc[1] / (final.iloc[0] + final.iloc[1]) * 100
deg_rate_arr = np.asarray(degrad_rate)

In [ ]:
fig = px.line(x=[i for i in range(len(deg_rate_arr))], y=deg_rate_arr)
fig.show()

In [ ]:

print(final)

In [1]:
from nbformat import read, write
def Remove_Output(Book):

    for cell in Book.cells:

        if hasattr(cell, "outputs"):

            cell.outputs = []

        if hasattr(cell, "prompt_number"):

            del cell["prompt_number"]

Book = read(open("sandbox2_2.ipynb"), 4)
Remove_Output(Book)
write(Book, open("sandbox2_2.ipynb", "w"), 4)